<a href="https://www.kaggle.com/code/marslanarshad/car-accident-detection-yolov5?scriptVersionId=252766382" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# 🚀 Step 1: Clone your YOLOv5 repo (already cloned in your case)
# If not cloned, uncomment the next line:
# !git clone https://github.com/M-ArslanArshad/car-accident-yolo.git

!git clone https://github.com/M-ArslanArshad/car-accident-yolo.git


Cloning into 'car-accident-yolo'...
remote: Enumerating objects: 225, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 225 (delta 76), reused 186 (delta 47), pack-reused 0 (from 0)
Receiving objects: 100% (225/225), 3.63 MiB | 11.54 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [2]:
# ✅ Step 2: Navigate to the directory
%cd /kaggle/working/car-accident-yolo


/kaggle/working/car-accident-yolo


In [3]:
# ✅ Step 3: Install required dependencies
!pip install -r requirements.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia

In [4]:

# ✅ Step 4: (Optional) Check available GPU
import torch
print("CUDA Available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


CUDA Available: True
Device: Tesla T4


In [ ]:

# ✅ Step 5: Prepare Dataset YAML file
# Ensure your dataset is already uploaded in this format:
# - images/train/
# - images/val/
# - labels/train/
# - labels/val/
# Your dataset YAML file (e.g. data.yaml) should point to correct paths.

# ✅ Example `data.yaml` content (replace with your class names and paths)


In [11]:
custom_yaml_path = "/kaggle/working/car-accident-yolo/data/custom_coco.yaml"

new_yaml_content = """# YOLOv5 🚀 by Ultralytics, AGPL-3.0 license
# COCO128 dataset https://www.kaggle.com/ultralytics/coco128 (first 128 images from COCO train2017) by Ultralytics
# Example usage: python train.py --data coco128.yaml
# parent
# ├── yolov5
# └── datasets
#     └── coco128  ← downloads here (7 MB)


# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: ../kaggle/input/car-accidents-and-deformation-datasetannotated/custom_dataset  # dataset root dir
train:  /kaggle/input/car-accidents-and-deformation-datasetannotated/custom_dataset/images/train # train images (relative to 'path') 128 images
val: /kaggle/input/car-accidents-and-deformation-datasetannotated/custom_dataset/images/val  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
names:
  0: no_accident
  1: accident_30
  2: accident_50
  3: accident_70
  4: accident_100
 
 

# Download script/URL (optional)
#download: https://ultralytics.com/assets/coco128.zip
"""
with open(custom_yaml_path, "w") as f:
    f.write(new_yaml_content.strip())

print("✅ custom_coco.yaml has been updated.")


✅ custom_coco.yaml has been updated.


**tweak the hyperparameters to get the best results**

In [18]:
# Inject weights_only=False into train.py (only once needed)
file_path = "/kaggle/working/car-accident-yolo/train.py"

with open(file_path, "r") as f:
    lines = f.readlines()

for i, line in enumerate(lines):
    if "torch.load(weights" in line and "weights_only" not in line:
        # Add weights_only=False to torch.load call
        lines[i] = line.replace("torch.load(weights", "torch.load(weights, weights_only=False")

with open(file_path, "w") as f:
    f.writelines(lines)

print("✅ Patched train.py with weights_only=False")


✅ Patched train.py with weights_only=False


In [ ]:
!python /kaggle/working/car-accident-yolo/train.py --img 640 --batch 16 --epochs 50 --data /kaggle/working/car-accident-yolo/data/custom_coco.yaml --weights yolov5s.pt --name car_accident_model


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-07-27 12:28:44.014443: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753619324.037259     327 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753619324.044301     327 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
wandb: WARNING If you're specifying your api key in code, ensure this code is 

In [ ]:
# ✅ Step 7: Save best.pt or last.pt to Kaggle output
import shutil
shutil.copy('runs/train/car_accident_model/weights/best.pt', '/kaggle/working/best.pt')
shutil.copy('runs/train/car_accident_model/weights/last.pt', '/kaggle/working/last.pt')

print("✅ Model training complete. Weights saved as best.pt and last.pt")


**Detection**

In [10]:
# ✅ Step 1: Change to the repo directory (if not already there)
%cd /kaggle/working/car-accident-yolo

# ✅ Step 2: Path to weights file (use best.pt or last.pt as needed)
weights_path = "/kaggle/working/car-accident-yolo/last.pt"  # or 'last.pt'

# ✅ Step 3: Provide path to images you want to test
# You can change this to any directory with images
#please add test images on your own .
test_images_path = "/kaggle/working/car-accident-yolo/data/images/bus.jpg"



/kaggle/working/car-accident-yolo


In [ ]:
# ✅ Step 4: Run detection
!python detect.py --weights {weights_path} --img 640 --conf 0.25 --source {test_images_path} --save-txt --save-conf
